In [1]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from numpy import pi,sqrt,exp,cos,sin,array
from numpy.linalg import inv
from lib.Model import *
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 13,
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage{amsfonts}",
    "lines.linewidth": 1
})

In [16]:
def mkimg(k=(1.0,1.0,1.0),m=(1.0,1.0),dt=0.01,t_end=40,x1_i=1.0,x2_i=1.0,plot=False):
    # numerical solution
    
    (t, x1,x2,v1,v2) = Model(k=(k1, k2, k3),m=(m1, m2),dt=dt,t_end=t_end,x1_i=x1_i,x2_i=x2_i,v1_i=0,v2_i=0)
    
    # Eigenmode expansion
    trF  = -(m2*k1 + (m1+m2)*k2 + m1*k3)/(m1*m2)
    detF =  (k1*k2 + k2*k3 + k3*k1)/(m1*m2)
    (lda1,lda2) = ((trF +sqrt(trF*trF-4*detF))/2, (trF -sqrt(trF*trF-4*detF))/2)
    (omega1,omega2) = (sqrt(-lda1),sqrt(-lda2)) 
    (mu1,mu2) = (array([[m2*lda1+(k2+k3)], [k2]]),array([[m2*lda2+(k2+k3)], [k2]]))
    M = np.hstack([mu1,mu2])
    invM = inv(M)
    C = invM@array([[x1_i],[x2_i]])
    (C1,C2) = (C[0][0],C[1][0])
    psi = C1*cos(omega1*t)*mu1 + C2*cos(omega2*t)*mu2
    (psi1,psi2) = (psi[0],psi[1])

    step1 = 30
    # --- plot -----------------------------------------------------------------------------
    fig = plt.figure(figsize=(6,4),dpi=200)
    ax1 = fig.add_subplot(2,1,1)
    ax2 = fig.add_subplot(2,1,2)
    marker1 = '--'
    marker2 = '-'
    zorder1 = 2
    zorder2 = 1
    linewidth1 = 1.5
    linewidth2 = 1
    ax1.plot(t,x1,marker1   ,color='r', label='numerical solution of $x_1$',  linewidth=linewidth1,zorder=zorder1)
    ax1.plot(t,psi1,marker2 ,color='k', label='eigenmode expansion of $x_2$', linewidth=linewidth2,zorder=zorder2)

    ax2.plot(t,x2,marker1   ,color='b', label='numerical solution of $x_2$',  linewidth=linewidth1,zorder=zorder1)
    ax2.plot(t,psi2,marker2 ,color='k', label='eigenmode expansion of $x_2$', linewidth=linewidth2,zorder=zorder2)

    [[[ax1,ax2][i].set_xlabel("$t$"),[ax1,ax2][i].set_ylabel(f"$x_{i+1}$")] for i in [0,1]]
    ax1.legend(loc='upper right')
    ax2.legend(loc='upper right')
    fig.tight_layout()
    if plot:
        plt.show()
    else:
        plt.close()
    name = f"(k1,k2,k3)=({k1},{k2},{k3}),(m1,m2)=({m1},{m2}),(x1_i,x1_2)=({x1_i},{x2_i}).pdf"
    fig.savefig("img/"+name)
    print("save image as :", name)

def print2LaTex(file=None):
    imgs = next(os.walk("./img"), (None, None, []))[2]
    for img in imgs:
        matches = re.findall(r'\((.*?)\)', img)
        if(img[-4:]!=".pdf"):
            continue
        (k1,k2,k3)  = matches[1].split(',')
        (m1,m2)     = matches[3].split(',')
        (x1_i,x1_2) = matches[5].split(',')
        matches = re.findall(r'\((.*?)\)', img)
        print(r"""
        \begin{figure}[H]
            \centering
            \includegraphics[width=0.9\linewidth]{../Numerical-Calculation/img/%s}
            \caption{Motion curve for $\left(k_1,k_2,k_3\right)=\left(%s,%s,%s\right)$, $\left(m_1,m_2\right)=\left(%s,%s\right)$ and the intial consition $\left(x_{1}(0),x_2(0)\right)=\left(%s,%s\right)$}
        \end{figure}"""%(img,k1,k2,k3,m1,m2,x1_i,x2_i),file=file)

In [17]:
"""
    (k1, k2, k3),  (m1, m2),  (x1_i,x2_i)
"""
!rm ./img/*.pdf
BCs = [
    ((1.0, 1.0, 1.0), (1.0, 1.0), (1.0, 1.0)),
    ((1.0, 1.0, 1.0), (1.0, 1.0), (1.0, -1.0)),
    ((4.0, 1.0, 4.0), (0.5, 0.5), (0.0, 1.0)),
    ((2.0, 0.5, 3.0), (1.0, 1.0), (1.0, -1.0)),
]
for BC in BCs:
    ((k1, k2, k3),(m1, m2),(x1_i,x2_i)) = BC
    mkimg(k=(k1, k2, k3),m=(m1, m2),dt=0.01,t_end=30,x1_i=x1_i,x2_i=x2_i)

file = open("./../Theoretical-Calculation/numerical-result.tex", 'w')
print2LaTex(file=file)
file.close()

save image as : (k1,k2,k3)=(1.0,1.0,1.0),(m1,m2)=(1.0,1.0),(x1_i,x1_2)=(1.0,1.0).pdf
save image as : (k1,k2,k3)=(1.0,1.0,1.0),(m1,m2)=(1.0,1.0),(x1_i,x1_2)=(1.0,-1.0).pdf
save image as : (k1,k2,k3)=(4.0,1.0,4.0),(m1,m2)=(0.5,0.5),(x1_i,x1_2)=(0.0,1.0).pdf
save image as : (k1,k2,k3)=(2.0,0.5,3.0),(m1,m2)=(1.0,1.0),(x1_i,x1_2)=(1.0,-1.0).pdf
